# 만들어진 아루코 마커를 사용해보자

In [1]:
### 아루코 마커 검출

In [2]:
import cv2
import numpy as np
import time

In [3]:
# 카메라 생성 및 카메라, 렌즈 파라메터 정의
cam = cv2.VideoCapture(0)  # 0은 기본 웹캠을 의미합니다.

# aruco detector 생성
board_type=cv2.aruco.DICT_6X6_250
marker_size = 400

arucoDict  = cv2.aruco.getPredefinedDictionary(board_type);
parameters = cv2.aruco.DetectorParameters()
detector   = cv2.aruco.ArucoDetector(arucoDict, parameters);

In [4]:
# 마커사이즈 실제 사이즈와 공간 좌표(x,y,z) -> z가 0인 이유는 마커가 평면이기 때문이다.
marker_3d_edges = np.array([    [0,0,0],
                                [0,marker_size,0],
                                [marker_size,marker_size,0],
                                [marker_size,0,0]], dtype = 'float32').reshape((4,1,3))


In [5]:
while True:
    # 웹캠으로부터 촬영 이미지 가져오기
    ret, img = cam.read()

    # 파란색상 정의
    blue_BGR = (255, 0, 0)

    if ret:
        # 마커(marker) 검출
        corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=parameters)

        # 검출된 마커들의 꼭지점을 이미지에 그려 확인
        if corners:
            cv2.aruco.drawDetectedMarkers(img, corners, ids)

        cv2.imshow("img", img)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

### 카메라 왜곡(추정값)을 이용한 좌표 시각화
##### 실제 왜곡값은 카메라 캘리브레이션을 이용해서 추출 가능(아마도?)

In [4]:
import cv2
import numpy as np

# 카메라 생성
cam = cv2.VideoCapture(0)  # 0은 기본 웹캠을 의미합니다.

# 카메라 행렬과 왜곡 계수 추정
ret, img = cam.read()
height, width, _ = img.shape
cmtx = [[width, 0, width/2], [0, width, height/2], [0, 0, 1]]
dist = [0, 0, 0, 0, 0]

# aruco detector 생성
board_type = cv2.aruco.DICT_6X6_250
arucoDict  = cv2.aruco.getPredefinedDictionary(board_type)
parameters = cv2.aruco.DetectorParameters()

# 마커사이즈 실제 사이즈와 공간 좌표(x,y,z) -> z가 0인 이유는 마커가 평면이기 때문이다.
marker_size = 200
marker_3d_edges = np.array([    [0,0,0],
                                [0,marker_size,0],
                                [marker_size,marker_size,0],
                                [marker_size,0,0]], dtype = 'float32').reshape((4,1,3))

while True:
    # 웹캠으로부터 촬영 이미지 가져오기
    ret, img = cam.read()

    # 파란색상 정의
    blue_BGR = (255, 0, 0)

    if ret:
        # 마커(marker) 검출
        corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=parameters)

        # 검출된 마커들의 외곽선을 이미지에 그림
        if corners:
            cv2.aruco.drawDetectedMarkers(img, corners, ids)

            # PnP
            for corner in corners:
                ret, rvec, tvec = cv2.solvePnP(marker_3d_edges, corner, np.array(cmtx), np.array(dist))
                if ret:
                    x = round(tvec[0][0], 2)
                    y = round(tvec[1][0], 2)
                    z = round(tvec[2][0], 2)
                    rx = round(np.rad2deg(rvec[0][0]), 2)
                    ry = round(np.rad2deg(rvec[1][0]), 2)
                    rz = round(np.rad2deg(rvec[2][0]), 2)
                    # PnP 결과를 이미지에 그려 확인
                    text1 = f"{x},{y},{z}"
                    text2 = f"{rx},{ry},{rz}"
                    cv2.putText(img, text1, (int(corner[0][0][0]-10), int(corner[0][0][1]+10)), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 255, 0), 2)
                    cv2.putText(img, text2, (int(corner[0][0][0]-10), int(corner[0][0][1]+40)), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 255, 0), 2)

        cv2.imshow("img", img)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


### 아루코 마커를 이용한 거리 계산 코드

In [7]:
import cv2
import numpy as np

# 카메라 생성
cam = cv2.VideoCapture(0)  # 0은 기본 웹캠을 의미합니다.

# 카메라 행렬과 왜곡 계수 추정
ret, img = cam.read()
height, width, _ = img.shape
cmtx = [[width, 0, width/2], [0, width, height/2], [0, 0, 1]]
dist = [0, 0, 0, 0, 0]

# aruco detector 생성
board_type = cv2.aruco.DICT_6X6_250
arucoDict  = cv2.aruco.getPredefinedDictionary(board_type)
parameters = cv2.aruco.DetectorParameters()

# 마커사이즈 실제 사이즈와 공간 좌표(x,y,z) -> z가 0인 이유는 마커가 평면이기 때문이다.
marker_size = 200
marker_3d_edges = np.array([    [0,0,0],
                                [0,marker_size,0],
                                [marker_size,marker_size,0],
                                [marker_size,0,0]], dtype = 'float32').reshape((4,1,3))

while True:
    # 웹캠으로부터 촬영 이미지 가져오기
    ret, img = cam.read()

    if ret:
        # 마커(marker) 검출
        corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=parameters)

        # 검출된 마커들의 외곽선을 이미지에 그림
        if corners:
            cv2.aruco.drawDetectedMarkers(img, corners, ids)

            # PnP
            for corner in corners:
                ret, rvec, tvec = cv2.solvePnP(marker_3d_edges, corner, np.array(cmtx), np.array(dist))
                if ret:
                    # 거리 정보를 계산
                    distance = np.sqrt(tvec[0][0]**2 + tvec[1][0]**2 + tvec[2][0]**2)
                    distance = round(distance, 2)
                    # 거리 정보를 이미지에 그려 확인
                    text = f"Distance: {distance/100:.1f} cm"
                    cv2.putText(img, text, (int(np.mean(corner[:, 0, 0])), int(np.mean(corner[:, 0, 1]))), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 255, 0), 2)

        cv2.imshow("img", img)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()
